In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [91]:
source_dir = '../data/undesired_outcomes_pred/'
dest_dir = source_dir
extension = '.csv'

In [92]:
elog_uo = pd.read_csv(source_dir.__add__('elog_uo.csv'))

In [93]:
elog_uo.columns = elog_uo.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [94]:
pd.options.display.max_columns = 100

In [95]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [96]:
elog_uo.sample(4)

,case_id,activity,resource,complete_timestamp,case_amount_applied0,case_amount_applied1,case_amount_applied2,case_amount_applied3,case_applicant,case_area,case_basic_payment,case_cross_compliance,case_department,case_greening,case_number_parcels,case_payment_actual0,case_payment_actual1,case_payment_actual2,case_payment_actual3,case_penalty_abp,case_penalty_agp,case_penalty_ajlp,case_penalty_auvp,case_penalty_avbp,case_penalty_avgp,case_penalty_avjlp,case_penalty_avuvp,case_penalty_b16,case_penalty_b2,case_penalty_b3,case_penalty_b4,case_penalty_b5,case_penalty_b5f,case_penalty_b6,case_penalty_bgk,case_penalty_bgkv,case_penalty_bgp,case_penalty_c16,case_penalty_c4,case_penalty_c9,case_penalty_cc,case_penalty_gp1,case_penalty_jlp1,case_penalty_jlp2,case_penalty_jlp3,case_penalty_jlp5,case_penalty_jlp6,case_penalty_jlp7,case_penalty_v5,case_penalty_amount0,case_penalty_amount1,case_penalty_amount2,case_penalty_amount3,case_redistribution,case_rejected,case_risk_factor,case_selected_manually,case_selected_random,case_selected_risk,case_small_farmer,case_year,case_young_farmer,activity,docid,doctype,eventid,note,subprocess,success,num_changes_applied_amount,final_applied_amount,diff_applied_amount,applied_amount_lowered,applied_amount_changed,num_changes_actual_amount,final_actual_amount,diff_actual_amount,actual_amount_lowered,actual_amount_changed,actual_amount_pay,num_changes_penalty_amount,final_penalty_amount,penalty_amount_applied,num_penalty_reasons,num_severe_penalties,penalty_applied,severe_penalty_applied,timestamp_year,timestamp_month,undesired_outcome1,undesired_outcome2,undesired_outcome
24497,008b2c4af83714ed,Payment application-Application-finish editing,727350,2016-11-14 08:56:41.810,9679.96,NaN,NaN,NaN,14a23359946c773c,28.4699,True,0.0,4e,True,7,9683.17,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2016,False,finish editing,ff00055ed5749441,Payment application,fec005617e44e990,automatic,Application,True,0,9679.96,0.0,False,False,0,9683.17,0.0,False,False,False,0,0.0,False,0,0,False,False,2016,11,False,False,False
43662,87bb13ecaa14c672,Payment application-Application-finish editing,727350,2017-11-17 09:23:08.460,44363.01,NaN,NaN,NaN,71e4f2c55d135399,139.1062,True,0.0,6b,True,26,44349.04,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,True,finish editing,ff40057d15c08a67,Payment application,ffc0057f1aa2d930,automatic,Application,True,0,44363.01,0.0,False,False,0,44349.04,0.0,False,False,False,0,0.0,False,0,0,False,False,2017,11,True,False,True
31144,7f0fd5a30ee47aa2,Payment application-Application-finish editing,727350,2017-11-17 06:51:18.179,32945.57,NaN,NaN,NaN,d5ed6a84cd4300d1,112.9192,True,0.0,6b,True,39,32935.43,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,False,finish editing,ff40057d15b969e5,Payment application,ffc0057f19ece3bc,automatic,Application,True,0,32945.57,0.0,False,False,0,32935.43,0.0,False,False,False,0,0.0,False,0,0,False,False,2017,11,True,False,True
14751,49b0354176affca5,Payment application-Application-finish editing,727350,2016-12-02 08:52:26.817,22048.89,NaN,NaN,NaN,e942c1c89c750745,70.7263,True,0.0,4e,True,32,22057.32,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2016,False,finish editing,ff0

In [97]:
features = ['(case) amount_applied0', '(case) area', '(case) department', '(case) number_parcels', '(case) payment_actual0',
           '(case) penalty_amount0', '(case) penalty_B2', '(case) year', 'final applied amount',
             'final actual amount', 'final penalty amount', 'num_penalty_reasons', 'num_severe_penalties',
            'num changes actual amount', 'num changes applied amount', 'num changes penalty amount', 'penalty amount applied', 
            'penalty applied', 'severe penalty applied', 'timestamp_month', 'timestamp_year']
features = [x.strip().lower().replace(' ', '_').replace('(', '').replace(')', '') for x in features]

In [98]:
numeric_feat = ['(case) amount_applied0', '(case) area', '(case) year', '(case) number_parcels', '(case) payment_actual0',
                '(case) penalty_amount0', 'final applied amount',  'final actual amount', 'final penalty amount',
                'num_penalty_reasons', 'num_severe_penalties', 'num changes actual amount', 'num changes applied amount',
                'num changes penalty amount', 'timestamp_month', 'timestamp_year', '(case) penalty_B2', 'penalty amount applied',
                'penalty applied', 'severe penalty applied'] 
numeric_feat = [x.strip().lower().replace(' ', '_').replace('(', '').replace(')', '') for x in numeric_feat]
cat_feat = list(set(features).difference(set(numeric_feat)))

In [99]:
elog_uo[['undesired_outcome']] = elog_uo[['undesired_outcome']].astype(bool)

In [100]:
elog_uo = elog_uo[features + ['undesired_outcome']]

In [101]:
elog_uo.sample(3)

,case_amount_applied0,case_area,case_department,case_number_parcels,case_payment_actual0,case_penalty_amount0,case_penalty_b2,case_year,final_applied_amount,final_actual_amount,final_penalty_amount,num_penalty_reasons,num_severe_penalties,num_changes_actual_amount,num_changes_applied_amount,num_changes_penalty_amount,penalty_amount_applied,penalty_applied,severe_penalty_applied,timestamp_month,timestamp_year,undesired_outcome
6004,448.99,1.3475,6b,1,448.99,0.0,False,2015,448.99,448.99,0.0,0,0,0,0,0,False,False,False,12,2015,False
33288,12504.41,38.4677,d4,14,12500.97,0.0,False,2017,12504.41,12500.97,0.0,0,0,0,0,0,False,False,False,11,2017,False
33054,1575.56,4.7409,d4,2,1575.56,0.0,False,2017,1575.56,1575.56,0.0,0,0,0,0,0,False,False,False,11,2017,False


In [102]:
elog_uo['case_department'].unique()

array(['e7', 'd4', '4e', '6b'], dtype=object)

In [103]:
department = tf.feature_column.categorical_column_with_vocabulary_list('case_department', ['e7', 'd4', '4e', '6b'])

In [104]:
case_amount_applied0 = tf.feature_column.numeric_column('case_amount_applied0')
case_area = tf.feature_column.numeric_column('case_area')
case_year = tf.feature_column.numeric_column('case_year')
case_number_parcels = tf.feature_column.numeric_column('case_number_parcels')
case_payment_actual0 = tf.feature_column.numeric_column('case_payment_actual0')
final_applied_amount = tf.feature_column.numeric_column('final_applied_amount')
final_actual_amount = tf.feature_column.numeric_column('final_actual_amount')
final_penalty_amount = tf.feature_column.numeric_column('final_penalty_amount')
num_penalty_reasons = tf.feature_column.numeric_column('num_penalty_reasons')
num_severe_penalties = tf.feature_column.numeric_column('num_severe_penalties')
num_changes_actual_amount = tf.feature_column.numeric_column('num_changes_actual_amount')
num_changes_applied_amount = tf.feature_column.numeric_column('num_changes_applied_amount')
num_changes_penalty_amount = tf.feature_column.numeric_column('num_changes_penalty_amount')
timestamp_month = tf.feature_column.numeric_column('timestamp_month')
timestamp_year = tf.feature_column.numeric_column('timestamp_year')
case_penalty_B2 = tf.feature_column.numeric_column('case_penalty_b2')
penalty_amount_applied = tf.feature_column.numeric_column('penalty_amount_applied')
penalty_applied = tf.feature_column.numeric_column('penalty_applied')
severe_penalty_applied = tf.feature_column.numeric_column('severe_penalty_applied')
num_feats_vars = [case_amount_applied0, case_area, case_year, case_number_parcels, case_payment_actual0, final_actual_amount,
                  final_applied_amount, final_penalty_amount, num_penalty_reasons, num_severe_penalties,
                  num_changes_actual_amount, num_changes_applied_amount, num_changes_penalty_amount, timestamp_month,
                  timestamp_year, case_penalty_B2, penalty_amount_applied, penalty_applied, severe_penalty_applied]

In [105]:
features_vars = num_feats_vars + [department]

In [106]:
elog_features = elog_uo.drop('undesired_outcome', axis=1)

In [107]:
target = elog_uo['undesired_outcome']

In [108]:
x_train, x_test, y_train, y_test = train_test_split(elog_features, target, test_size = 0.3)

In [109]:
input_f = tf.estimator.inputs.pandas_input_fn(x = x_train, y = y_train, batch_size = 100, 
                                              num_epochs = 1000, shuffle = True)

In [110]:
model = tf.estimator.LinearClassifier(feature_columns = features_vars, n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\s158881\\AppData\\Local\\Temp\\tmp60431s62', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [ ]:
model.train(input_fn = input_f, steps = int(1e6))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\s158881\AppData\Local\Temp\tmp60431s62\model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into C:\Users\s158881\AppData\Local\Temp\tmp60431s62\model.ckpt.
INFO:tensorflow:loss = 3231.21, step = 1001
INFO:tensorflow:global_step/sec: 268.788
INFO:tensorflow:loss = 3301.85, step = 1101 (0.375 sec)
INFO:tensorflow:global_step/sec: 309.381
INFO:tensorflow:loss = 16589.9, step = 1201 (0.323 sec)
INFO:tensorflow:global_step/sec: 308.411
INFO:tensorflow:loss = 7558.33, step = 1301 (0.325 sec)
INFO:tensorflow:global_step/sec: 305.581
INFO:tensorflow:loss = 20968.9, step = 1401 (0.327 sec)
INFO:tensorflow:global_step/sec: 309.548
INFO:tensorflow:loss = 4931.61, step = 1501 (0.324 sec)
INFO:tensorflow:global_step/sec: 309.391
INFO:tensorflow:loss = 19021.4, step = 1601 (0.323 sec)
INFO:tensorflow:global_step/sec: 311.735
INFO:tensorflow:loss = 14508.8, step = 1701 (0.321 sec)
INFO:tensorflow:gl

INFO:tensorflow:global_step/sec: 271.19
INFO:tensorflow:loss = 2200.85, step = 9301 (0.368 sec)
INFO:tensorflow:global_step/sec: 271.549
INFO:tensorflow:loss = 4458.09, step = 9401 (0.368 sec)
INFO:tensorflow:global_step/sec: 276.399
INFO:tensorflow:loss = 5508.73, step = 9501 (0.362 sec)
INFO:tensorflow:global_step/sec: 269.311
INFO:tensorflow:loss = 1934.75, step = 9601 (0.372 sec)
INFO:tensorflow:global_step/sec: 252.994
INFO:tensorflow:loss = 4727.11, step = 9701 (0.395 sec)
INFO:tensorflow:global_step/sec: 263.129
INFO:tensorflow:loss = 1550.3, step = 9801 (0.380 sec)
INFO:tensorflow:global_step/sec: 262.277
INFO:tensorflow:loss = 4208.7, step = 9901 (0.381 sec)
INFO:tensorflow:global_step/sec: 269.699
INFO:tensorflow:loss = 372.173, step = 10001 (0.371 sec)
INFO:tensorflow:global_step/sec: 272.271
INFO:tensorflow:loss = 8378.77, step = 10101 (0.367 sec)
INFO:tensorflow:global_step/sec: 258.187
INFO:tensorflow:loss = 3281.51, step = 10201 (0.387 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:global_step/sec: 273.782
INFO:tensorflow:loss = 982.295, step = 17701 (0.365 sec)
INFO:tensorflow:global_step/sec: 290.916
INFO:tensorflow:loss = 7277.06, step = 17801 (0.343 sec)
INFO:tensorflow:global_step/sec: 289.64
INFO:tensorflow:loss = 6415.03, step = 17901 (0.345 sec)
INFO:tensorflow:global_step/sec: 292.574
INFO:tensorflow:loss = 2255.97, step = 18001 (0.342 sec)
INFO:tensorflow:global_step/sec: 189.259
INFO:tensorflow:loss = 2563.34, step = 18101 (0.530 sec)
INFO:tensorflow:global_step/sec: 216.532
INFO:tensorflow:loss = 730.722, step = 18201 (0.459 sec)
INFO:tensorflow:global_step/sec: 220.84
INFO:tensorflow:loss = 957.472, step = 18301 (0.454 sec)
INFO:tensorflow:global_step/sec: 255.249
INFO:tensorflow:loss = 1336.1, step = 18401 (0.392 sec)
INFO:tensorflow:global_step/sec: 267.533
INFO:tensorflow:loss = 1465.54, step = 18501 (0.374 sec)
INFO:tensorflow:global_step/sec: 230.236
INFO:tensorflow:loss = 478.716, step = 18601 (0.435 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 234.569
INFO:tensorflow:loss = 1645.57, step = 26101 (0.434 sec)
INFO:tensorflow:global_step/sec: 179.562
INFO:tensorflow:loss = 3385.03, step = 26201 (0.553 sec)
INFO:tensorflow:global_step/sec: 193.087
INFO:tensorflow:loss = 3299.34, step = 26301 (0.516 sec)
INFO:tensorflow:global_step/sec: 258.202
INFO:tensorflow:loss = 3365.32, step = 26401 (0.386 sec)
INFO:tensorflow:global_step/sec: 260.553
INFO:tensorflow:loss = 6062.84, step = 26501 (0.385 sec)
INFO:tensorflow:global_step/sec: 237.356
INFO:tensorflow:loss = 2439.84, step = 26601 (0.421 sec)
INFO:tensorflow:global_step/sec: 146.843
INFO:tensorflow:loss = 5489.76, step = 26701 (0.684 sec)
INFO:tensorflow:global_step/sec: 215.36
INFO:tensorflow:loss = 1048.62, step = 26801 (0.461 sec)
INFO:tensorflow:global_step/sec: 133.673
INFO:tensorflow:loss = 1693.41, step = 26901 (0.752 sec)
INFO:tensorflow:global_step/sec: 124.056
INFO:tensorflow:loss = 4253.39, step = 27001 (0.801 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 297.413
INFO:tensorflow:loss = 1086.08, step = 34501 (0.337 sec)
INFO:tensorflow:global_step/sec: 297.832
INFO:tensorflow:loss = 1628.69, step = 34601 (0.336 sec)
INFO:tensorflow:global_step/sec: 283.876
INFO:tensorflow:loss = 3533.85, step = 34701 (0.352 sec)
INFO:tensorflow:global_step/sec: 301.854
INFO:tensorflow:loss = 780.887, step = 34801 (0.331 sec)
INFO:tensorflow:global_step/sec: 301.451
INFO:tensorflow:loss = 812.598, step = 34901 (0.332 sec)
INFO:tensorflow:global_step/sec: 288.803
INFO:tensorflow:loss = 1201.37, step = 35001 (0.347 sec)
INFO:tensorflow:global_step/sec: 264.698
INFO:tensorflow:loss = 1627.44, step = 35101 (0.379 sec)
INFO:tensorflow:global_step/sec: 257.2
INFO:tensorflow:loss = 1644.61, step = 35201 (0.386 sec)
INFO:tensorflow:global_step/sec: 281.475
INFO:tensorflow:loss = 785.784, step = 35301 (0.357 sec)
INFO:tensorflow:global_step/sec: 299.628
INFO:tensorflow:loss = 1069.6, step = 35401 (0.333 sec)
INFO:tensorflow:global_

...

INFO:tensorflow:global_step/sec: 133.869
INFO:tensorflow:loss = 147.245, step = 259301 (0.745 sec)
INFO:tensorflow:global_step/sec: 178.594
INFO:tensorflow:loss = 740.862, step = 259401 (0.560 sec)
INFO:tensorflow:global_step/sec: 191.994
INFO:tensorflow:loss = 397.17, step = 259501 (0.519 sec)
INFO:tensorflow:global_step/sec: 155.296
INFO:tensorflow:loss = 417.003, step = 259601 (0.646 sec)
INFO:tensorflow:global_step/sec: 198.872
INFO:tensorflow:loss = 1495.56, step = 259701 (0.503 sec)
INFO:tensorflow:global_step/sec: 236.527
INFO:tensorflow:loss = 493.84, step = 259801 (0.420 sec)
INFO:tensorflow:global_step/sec: 176.082
INFO:tensorflow:loss = 292.299, step = 259901 (0.570 sec)
INFO:tensorflow:global_step/sec: 215.373
INFO:tensorflow:loss = 220.188, step = 260001 (0.465 sec)
INFO:tensorflow:global_step/sec: 178.931
INFO:tensorflow:loss = 369.498, step = 260101 (0.562 sec)
INFO:tensorflow:global_step/sec: 172.596
INFO:tensorflow:loss = 122.153, step = 260201 (0.573 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 192.548
INFO:tensorflow:loss = 629.524, step = 267601 (0.524 sec)
INFO:tensorflow:global_step/sec: 168.806
INFO:tensorflow:loss = 1142.03, step = 267701 (0.590 sec)
INFO:tensorflow:global_step/sec: 200.738
INFO:tensorflow:loss = 726.533, step = 267801 (0.499 sec)
INFO:tensorflow:global_step/sec: 164.094
INFO:tensorflow:loss = 791.393, step = 267901 (0.607 sec)
INFO:tensorflow:global_step/sec: 146.208
INFO:tensorflow:loss = 228.843, step = 268001 (0.686 sec)
INFO:tensorflow:global_step/sec: 167.673
INFO:tensorflow:loss = 199.835, step = 268101 (0.594 sec)
INFO:tensorflow:global_step/sec: 184.549
INFO:tensorflow:loss = 245.019, step = 268201 (0.546 sec)
INFO:tensorflow:global_step/sec: 217.957
INFO:tensorflow:loss = 182.103, step = 268301 (0.457 sec)
INFO:tensorflow:global_step/sec: 236.249
INFO:tensorflow:loss = 2459.9, step = 268401 (0.423 sec)
INFO:tensorflow:global_step/sec: 198.872
INFO:tensorflow:loss = 201.444, step = 268501 (0.500 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 246.141
INFO:tensorflow:loss = 1377.05, step = 275901 (0.406 sec)
INFO:tensorflow:global_step/sec: 293.49
INFO:tensorflow:loss = 214.002, step = 276001 (0.341 sec)
INFO:tensorflow:global_step/sec: 302.36
INFO:tensorflow:loss = 162.301, step = 276101 (0.330 sec)
INFO:tensorflow:global_step/sec: 293.451
INFO:tensorflow:loss = 414.249, step = 276201 (0.341 sec)
INFO:tensorflow:global_step/sec: 263.337
INFO:tensorflow:loss = 773.176, step = 276301 (0.387 sec)
INFO:tensorflow:global_step/sec: 255.91
INFO:tensorflow:loss = 287.968, step = 276401 (0.384 sec)
INFO:tensorflow:global_step/sec: 285.057
INFO:tensorflow:loss = 276.275, step = 276501 (0.350 sec)
INFO:tensorflow:global_step/sec: 271.53
INFO:tensorflow:loss = 673.91, step = 276601 (0.369 sec)
INFO:tensorflow:global_step/sec: 255.256
INFO:tensorflow:loss = 92.6947, step = 276701 (0.392 sec)
INFO:tensorflow:global_step/sec: 294.763
INFO:tensorflow:loss = 222.323, step = 276801 (0.339 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 146.532
INFO:tensorflow:loss = 982.083, step = 284101 (0.682 sec)
INFO:tensorflow:global_step/sec: 160.535
INFO:tensorflow:loss = 430.509, step = 284201 (0.626 sec)
INFO:tensorflow:global_step/sec: 171.707
INFO:tensorflow:loss = 1841.63, step = 284301 (0.580 sec)
INFO:tensorflow:global_step/sec: 206.047
INFO:tensorflow:loss = 491.895, step = 284401 (0.486 sec)
INFO:tensorflow:global_step/sec: 253.628
INFO:tensorflow:loss = 352.778, step = 284501 (0.393 sec)
INFO:tensorflow:global_step/sec: 239.574
INFO:tensorflow:loss = 221.991, step = 284601 (0.417 sec)
INFO:tensorflow:global_step/sec: 245.233
INFO:tensorflow:loss = 146.801, step = 284701 (0.408 sec)
INFO:tensorflow:global_step/sec: 297.42
INFO:tensorflow:loss = 163.489, step = 284801 (0.337 sec)
INFO:tensorflow:global_step/sec: 280.269
INFO:tensorflow:loss = 357.237, step = 284901 (0.356 sec)
INFO:tensorflow:global_step/sec: 252.356
INFO:tensorflow:loss = 677.475, step = 285001 (0.396 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 238.334
INFO:tensorflow:loss = 247.655, step = 292401 (0.419 sec)
INFO:tensorflow:global_step/sec: 206.693
INFO:tensorflow:loss = 316.199, step = 292501 (0.484 sec)
INFO:tensorflow:global_step/sec: 228.581
INFO:tensorflow:loss = 775.106, step = 292601 (0.441 sec)
INFO:tensorflow:global_step/sec: 241.362
INFO:tensorflow:loss = 301.083, step = 292701 (0.412 sec)
INFO:tensorflow:global_step/sec: 243.153
INFO:tensorflow:loss = 1852.71, step = 292801 (0.411 sec)
INFO:tensorflow:global_step/sec: 255.932
INFO:tensorflow:loss = 145.063, step = 292901 (0.391 sec)
INFO:tensorflow:global_step/sec: 111.2
INFO:tensorflow:loss = 268.342, step = 293001 (0.898 sec)
INFO:tensorflow:global_step/sec: 132.187
INFO:tensorflow:loss = 1507.18, step = 293101 (0.759 sec)
INFO:tensorflow:global_step/sec: 175.321
INFO:tensorflow:loss = 220.684, step = 293201 (0.567 sec)
INFO:tensorflow:global_step/sec: 220.118
INFO:tensorflow:loss = 908.857, step = 293301 (0.456 sec)
INFO:tensorf

In [ ]:
evaluation_model_function = tf.estimator.inputs.pandas_input_fn(x = x_test, y = y_test, batch_size = 64,
                                                               num_epochs = 10, shuffle = False)

In [ ]:
results = model.evaluate(evaluation_model_function)
results